# Imports 

In [1]:
import pandas as pd
import numpy as np
import os

import plotly.express as px

from math import ceil
from datetime import datetime

In [2]:
# def show_named_plotly_colours():
#     """
#     function to display to user the colours to match plotly's named
#     css colours.

#     Reference:
#         #https://community.plotly.com/t/plotly-colours-list/11730/3

#     Returns:
#         plotly dataframe with cell colour to match named colour name

#     """
#     s='''
#         aliceblue, antiquewhite, aqua, aquamarine, azure,
#         beige, bisque, black, blanchedalmond, blue,
#         blueviolet, brown, burlywood, cadetblue,
#         chartreuse, chocolate, coral, cornflowerblue,
#         cornsilk, crimson, cyan, darkblue, darkcyan,
#         darkgoldenrod, darkgray, darkgrey, darkgreen,
#         darkkhaki, darkmagenta, darkolivegreen, darkorange,
#         darkorchid, darkred, darksalmon, darkseagreen,
#         darkslateblue, darkslategray, darkslategrey,
#         darkturquoise, darkviolet, deeppink, deepskyblue,
#         dimgray, dimgrey, dodgerblue, firebrick,
#         floralwhite, forestgreen, fuchsia, gainsboro,
#         ghostwhite, gold, goldenrod, gray, grey, green,
#         greenyellow, honeydew, hotpink, indianred, indigo,
#         ivory, khaki, lavender, lavenderblush, lawngreen,
#         lemonchiffon, lightblue, lightcoral, lightcyan,
#         lightgoldenrodyellow, lightgray, lightgrey,
#         lightgreen, lightpink, lightsalmon, lightseagreen,
#         lightskyblue, lightslategray, lightslategrey,
#         lightsteelblue, lightyellow, lime, limegreen,
#         linen, magenta, maroon, mediumaquamarine,
#         mediumblue, mediumorchid, mediumpurple,
#         mediumseagreen, mediumslateblue, mediumspringgreen,
#         mediumturquoise, mediumvioletred, midnightblue,
#         mintcream, mistyrose, moccasin, navajowhite, navy,
#         oldlace, olive, olivedrab, orange, orangered,
#         orchid, palegoldenrod, palegreen, paleturquoise,
#         palevioletred, papayawhip, peachpuff, peru, pink,
#         plum, powderblue, purple, red, rosybrown,
#         royalblue, saddlebrown, salmon, sandybrown,
#         seagreen, seashell, sienna, silver, skyblue,
#         slateblue, slategray, slategrey, snow, springgreen,
#         steelblue, tan, teal, thistle, tomato, turquoise,
#         violet, wheat, white, whitesmoke, yellow,
#         yellowgreen
#         '''
#     li=s.split(',')
#     li=[l.replace('\n','') for l in li]
#     li=[l.replace(' ','') for l in li]

#     import pandas as pd
#     import plotly.graph_objects as go

#     df=pd.DataFrame.from_dict({'colour': li})
#     fig = go.Figure(data=[go.Table(
#       header=dict(
#         values=["Plotly Named CSS colours"],
#         line_color='black', fill_color='white',
#         align='center', font=dict(color='black', size=14)
#       ),
#       cells=dict(
#         values=[df.colour],
#         line_color=[df.colour], fill_color=[df.colour],
#         align='center', font=dict(color='black', size=11)
#       ))
#     ])

#     fig.show()
# show_named_plotly_colours()

In [3]:
datetime.now().year

2022

In [4]:
os.getcwd()

'/mnt/d/dev/app_prenoms/code'

In [5]:
data_nat = '/mnt/d/dev/app_prenoms/data/nat2020.csv'

In [6]:
df_raw = pd.read_csv(data_nat, sep=';')

# Observations et nettoyage

In [8]:
df_raw.head()

,sexe,preusuel,annais,nombre
0,1,_PRENOMS_RARES,1900,1250
1,1,_PRENOMS_RARES,1901,1342
2,1,_PRENOMS_RARES,1902,1330
3,1,_PRENOMS_RARES,1903,1286
4,1,_PRENOMS_RARES,1904,1430


In [13]:
df_raw[df_raw.sexe == 1].groupby('preusuel').agg(nombre=('nombre', sum)).sort_values(by=['nombre'], ascending=False).head()

,nombre
preusuel,
JEAN,1914060
PIERRE,891611
MICHEL,820353
_PRENOMS_RARES,799188
ANDRÉ,711950


In [9]:
df_raw.describe()

,sexe,nombre
count,667364.000000,667364.000000
mean,1.543828,129.772665
std,0.498076,884.912504
min,1.000000,1.000000
25%,1.000000,4.000000
50%,2.000000,8.000000
75%,2.000000,25.000000
max,2.000000,53584.000000


In [10]:
# Nombre de prénoms uniques
df_raw.preusuel.unique().shape

(35011,)

In [11]:
df_raw[df_raw.annais == 'XXXX']

,sexe,preusuel,annais,nombre
123,1,A,XXXX,21
130,1,AADAM,XXXX,9
138,1,AADEL,XXXX,28
164,1,AADIL,XXXX,34
169,1,AAHIL,XXXX,6
...,...,...,...,...
667297,2,ZÜMRA,XXXX,1
667315,2,ZUZANNA,XXXX,10
667336,2,ZYA,XXXX,6
667346,2,ZYNA,XXXX,16


Conditions portant sur les prénoms retenus

    1. Sur la période allant de 1900 à 1945, le prénom a été attribué au moins 20 fois à des personnes de sexe féminin et/ou au moins 20 fois à des personnes de sexe masculin
    2. Sur la période allant de 1946 à 2020, le prénom a été attribué au moins 20 fois à des personnes de sexe féminin et/ou au moins 20 fois à des personnes de sexe masculin
    3. Pour une année de naissance donnée, le prénom a été attribué au moins 3 fois à des personnes de sexe féminin ou de sexe masculin

Les effectifs des prénoms ne remplissant pas les conditions 1 et 2 sont regroupés (pour chaque sexe et chaque année de naissance) dans un enregistrement dont le champ prénom (PREUSUEL) prend la valeur «_PRENOMS_RARES_». Les effectifs des prénoms remplissant la condition 2 mais pas la condition 3 sont regroupés (pour chaque sexe et chaque prénom) dans un enregistrement dont le champ année de naissance (ANNAIS) prend la valeur «XXXX».

In [12]:
# On supprime les valeurs de 'annais' == 'XXXX'
df_raw = df_raw[df_raw.annais != 'XXXX'].copy()
df_raw.annais = df_raw.annais.astype(int)
df_raw.describe()

,sexe,annais,nombre
count,630689.000000,630689.000000,630689.000000
mean,1.543861,1983.349118,135.979288
std,0.498073,31.492272,909.887013
min,1.000000,1900.000000,3.000000
25%,1.000000,1965.000000,4.000000
50%,2.000000,1993.000000,8.000000
75%,2.000000,2009.000000,24.000000
max,2.000000,2020.000000,53584.000000


In [13]:
df_raw.preusuel.sort_values()

121           A
122           A
124       AADAM
125       AADAM
126       AADAM
          ...  
588418    ÖZLEM
650267     ÜMMÜ
650268     ÜMMÜ
650269     ÜMMÜ
563597      NaN
Name: preusuel, Length: 630689, dtype: object

In [14]:
df_raw[df_raw.preusuel.isna()]

,sexe,preusuel,annais,nombre
563597,2,NaN,2003,3


In [15]:
df_raw.dropna(inplace=True)

In [16]:
df_raw.preusuel = df_raw.preusuel.apply(lambda x: x.lower())
df_raw.sample(5)

,sexe,preusuel,annais,nombre
113686,1,hani,1988,4
33640,1,aydine,2015,7
35901,1,badra,2009,3
665247,2,zely,2007,10
393728,2,elham,2018,4


In [17]:
df_raw.preusuel.unique().shape

(33098,)

In [18]:
df_raw[df_raw.preusuel == 'estelle']

,sexe,preusuel,annais,nombre
406192,2,estelle,1900,164
406193,2,estelle,1901,191
406194,2,estelle,1902,153
406195,2,estelle,1903,153
406196,2,estelle,1904,151
...,...,...,...,...
406308,2,estelle,2016,244
406309,2,estelle,2017,212
406310,2,estelle,2018,195
406311,2,estelle,2019,158


In [19]:
df_nat = df_raw.copy(deep=True)

## Suppression des accents

In [20]:
df_nat[df_nat.preusuel.isin(['noël', 'agnès', 'mickaël', 'éléanor', 'léon', 'ingrid'])].preusuel.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').unique()

array(['agnes', 'leon', 'mickael', 'noel', 'eleanor', 'ingrid'],
      dtype=object)

In [21]:
df_nat['preusuel'] = df_nat['preusuel'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [22]:
df_nat.preusuel.unique().shape

(30133,)

In [23]:
df_nat.annais.max()

2020

## Aggrégation des prénoms désaccentués

In [26]:
df_gb_1 = df_nat.groupby(['preusuel', 'annais', 'sexe']).agg(nombre=('nombre', sum)).reset_index()
df_gb_1

,preusuel,annais,sexe,nombre
0,_prenoms_rares,1900,1,1250
1,_prenoms_rares,1900,2,1727
2,_prenoms_rares,1901,1,1342
3,_prenoms_rares,1901,2,1774
4,_prenoms_rares,1902,1,1330
...,...,...,...,...
604731,zyneb,2020,2,8
604732,zyon,2001,1,4
604733,zyon,2010,1,3
604734,zyon,2013,1,4


Vérification de l'aggrégation

In [27]:
df_raw.preusuel = df_raw.preusuel.apply(lambda x: x.lower())
df_raw[df_raw.preusuel.isin(['andrea', 'andréa'])].sort_values(by=['annais', 'sexe'])

,sexe,preusuel,annais,nombre
324952,2,andréa,1900,402
324953,2,andréa,1901,478
324954,2,andréa,1902,492
22100,1,andréa,1903,3
324955,2,andréa,1903,486
...,...,...,...,...
325071,2,andréa,2019,573
22098,1,andrea,2020,524
22179,1,andréa,2020,295
324950,2,andrea,2020,97


In [28]:
df_gb_1[df_gb_1.preusuel == 'andrea']

,preusuel,annais,sexe,nombre
38555,andrea,1900,2,402
38556,andrea,1901,2,478
38557,andrea,1902,2,492
38558,andrea,1903,1,3
38559,andrea,1903,2,486
...,...,...,...,...
38751,andrea,2018,2,708
38752,andrea,2019,1,687
38753,andrea,2019,2,691
38754,andrea,2020,1,819


In [29]:
df_gb_1.dtypes

preusuel    object
annais       int64
sexe         int64
nombre       int64
dtype: object

# Observation d'un prénom

In [30]:
sexe = 1
prenom = 'andrea'
df_prenom = df_gb_1[(df_gb_1.preusuel == prenom) & (df_gb_1.sexe == sexe)]
px.line(df_prenom, x='annais', y='nombre', markers=True)

In [31]:
seuil = 5000
annee = 2020
df_nat[(df_nat.annais == annee) & (df_nat.nombre <= seuil)].preusuel

129         aadam
168         aahil
177         aarav
235         aaron
254        aarone
           ...   
667296      zumra
667314    zuzanna
667335        zya
667345       zyna
667362      zyneb
Name: preusuel, Length: 13078, dtype: object

In [32]:
seuil = 5000
annee = 2020
df_nat[df_nat.annais == annee].sort_values(by='nombre')

,sexe,preusuel,annais,nombre
574959,2,neylla,2020,3
642010,2,taous,2020,3
469776,2,kaori,2020,3
92300,1,fily,2020,3
92309,1,finlay,2020,3
...,...,...,...,...
237467,1,raphael,2020,3970
98655,1,gabriel,2020,4415
169840,1,leo,2020,4496
120,1,_prenoms_rares,2020,26109


In [101]:
df_nat[df_nat.preusuel == 'sacha'].nombre.sum()

56927

In [113]:
df_nat.head()

,sexe,preusuel,annais,nombre
0,1,_prenoms_rares,1900,1250
1,1,_prenoms_rares,1901,1342
2,1,_prenoms_rares,1902,1330
3,1,_prenoms_rares,1903,1286
4,1,_prenoms_rares,1904,1430


# Totaux naissances et prénoms

In [33]:
df_total_nais_annees = df_nat.groupby('annais').agg({'nombre' : sum})
display(df_total_nais_annees.tail())
px.bar(df_total_nais_annees, y='nombre')

,nombre
annais,
2016,760502
2017,746478
2018,736262
2019,731117
2020,713273


In [35]:
df_total_prenoms_annees = df_nat[df_nat.sexe == 1].groupby('annais').agg({'preusuel' : 'count'})
display(df_total_prenoms_annees.tail())
px.bar(df_total_prenoms_annees, y='preusuel')

,preusuel
annais,
2016,6376
2017,6353
2018,6384
2019,6275
2020,6197


# Prénoms par périodes

## Récent 

In [37]:
nb_annees = 10
n_display = 50
df_last_years = df_nat[(df_nat.annais.isin([year for year in range(2021 - nb_annees, 2021)])) & (df_nat.sexe == 1)]
df_last_years.groupby(['preusuel']).agg({'nombre' : sum}).sort_values(by='nombre', ascending=False).head(n_display)

,nombre
preusuel,
_prenoms_rares,241526
gabriel,52332
leo,48692
lucas,47633
raphael,45944
louis,45935
jules,42799
hugo,41919
adam,41127


In [176]:
n_display = 50
df_last_3 = df_nat[(df_nat.annais.isin([year for year in range(2021 - 3, 2021)])) & (df_nat.sexe.isin([1, 2]))]
df_gb_3 = df_last_3.groupby(['preusuel']).agg(last_3=('nombre', sum)).sort_values(by='last_3', ascending=False).last_3.apply(lambda x: int(x / 3))
df_last_10 = df_nat[(df_nat.annais.isin([year for year in range(2021 - 10, 2021)])) & (df_nat.sexe.isin([1, 2]))]
df_gb_10 = df_last_10.groupby(['preusuel']).agg(last_10=('nombre', sum)).sort_values(by='last_10', ascending=False).last_10.apply(lambda x: int(x / 10))
df_compare = pd.concat((df_gb_10, df_gb_3), axis=1)
df_compare.head(n_display)

,last_10,last_3
preusuel,,
_prenoms_rares,50244,52740.0
gabriel,5233,4969.0
leo,4873,4789.0
lucas,4763,3657.0
emma,4625,3945.0
raphael,4594,4552.0
louis,4593,4061.0
jules,4279,3597.0
hugo,4191,3437.0


In [180]:
# Table rows colors
boyColor = 'lightskyblue'
girlColor = 'pink'
boyOrGirlColor = 'plum'
dict_colors = {1 : boyColor, 2 : girlColor, 3 : boyOrGirlColor}


df_prenoms_sexes = df_nat.groupby(['preusuel', 'sexe']).agg(nombre=('nombre', sum)).reset_index().sort_values(by=['preusuel', 'nombre'], ascending=False)
# display(df_prenoms_sexes[df_prenoms_sexes.preusuel.isin(['louis', 'hugo', 'camille'])])

df_prenoms_tot = df_prenoms_sexes.groupby('preusuel').agg(total=('nombre', sum)).reset_index()
# display(df_prenoms_tot[df_prenoms_tot.preusuel.isin(['louis', 'hugo', 'camille'])])

df_prenom = pd.merge(df_prenoms_tot, df_prenoms_sexes, on='preusuel')
# display(df_prenom[df_prenom.preusuel.isin(['louis', 'hugo', 'camille'])])

df_prenom['prop_sexe'] = df_prenom.nombre / df_prenom.total
df_prenom.drop_duplicates(subset=['preusuel'], inplace=True)

df_prenom['number_color'] = df_prenom.apply(lambda row: row.sexe if row.prop_sexe > 0.99 else 3, axis=1)
df_prenom['color'] = df_prenom.number_color.apply(lambda numcol: dict_colors[numcol])

display(df_prenom[df_prenom.preusuel.isin(['louis', 'hugo', 'camille', 'sacha', 'janne', 'emma', 'jade', 'alice'])])

dict_prenom_color = dict(df_prenom.loc[:, ['preusuel', 'color']].values)
dict_prenom_color

,preusuel,total,sexe,nombre,prop_sexe,number_color,color
1259,alice,198863,2,198817,0.999769,2,pink
4603,camille,282947,2,205213,0.725270,3,plum
8306,emma,162435,2,162326,0.999329,2,pink
11974,hugo,145770,1,145770,1.000000,1,lightskyblue
12988,jade,94466,2,94218,0.997375,2,pink
13197,janne,145,2,145,1.000000,2,pink
17600,louis,525425,1,525309,0.999779,1,lightskyblue
26177,sacha,56927,1,53561,0.940872,3,plum


{'_prenoms_rares': 'plum',
 'a': 'lightskyblue',
 'aadam': 'lightskyblue',
 'aadel': 'lightskyblue',
 'aadil': 'lightskyblue',
 'aahil': 'lightskyblue',
 'aakash': 'lightskyblue',
 'aaleyah': 'pink',
 'aalia': 'pink',
 'aaliya': 'pink',
 'aaliyah': 'pink',
 'aalya': 'pink',
 'aalyah': 'pink',
 'aanor': 'pink',
 'aarav': 'lightskyblue',
 'aaricia': 'pink',
 'aaron': 'lightskyblue',
 'aarone': 'lightskyblue',
 'aaronn': 'lightskyblue',
 'aaroun': 'lightskyblue',
 'aarush': 'lightskyblue',
 'aaya': 'pink',
 'aayan': 'lightskyblue',
 'aaziz': 'lightskyblue',
 'ab': 'lightskyblue',
 'ab-del': 'lightskyblue',
 'ababacar': 'lightskyblue',
 'abakar': 'lightskyblue',
 'abane': 'lightskyblue',
 'abas': 'lightskyblue',
 'abass': 'lightskyblue',
 'abasse': 'lightskyblue',
 'abassia': 'pink',
 'abbas': 'lightskyblue',
 'abbes': 'lightskyblue',
 'abbie': 'pink',
 'abbigaelle': 'pink',
 'abby': 'pink',
 'abby-gaelle': 'pink',
 'abbygael': 'pink',
 'abbygaelle': 'pink',
 'abbygail': 'pink',
 'abd': 'l

In [181]:
df_compare = df_compare.reset_index()
df_compare['color'] = df_compare.preusuel.apply(lambda prenom: dict_prenom_color[prenom])
df_compare.head(30)

,preusuel,last_10,last_3,color
0,_prenoms_rares,50244,52740.0,plum
1,gabriel,5233,4969.0,lightskyblue
2,leo,4873,4789.0,lightskyblue
3,lucas,4763,3657.0,lightskyblue
4,emma,4625,3945.0,pink
5,raphael,4594,4552.0,lightskyblue
6,louis,4593,4061.0,lightskyblue
7,jules,4279,3597.0,lightskyblue
8,hugo,4191,3437.0,lightskyblue
9,jade,4182,3877.0,pink


In [185]:
# Get number of sexes by Prénom
df_compare_prenoms = df_compare.reset_index()
df_compare_prenoms = df_compare_prenoms[['preusuel']]
df_compare_prenoms['color'] = df_compare_prenoms.preusuel.apply(lambda prenom: dict_prenom_color[prenom])
ar_colors = df_compare_prenoms.color.values.tolist()

[ar_colors * 5]

[['plum',
  'lightskyblue',
  'lightskyblue',
  'lightskyblue',
  'pink',
  'lightskyblue',
  'lightskyblue',
  'lightskyblue',
  'lightskyblue',
  'pink',
  'pink',
  'lightskyblue',
  'lightskyblue',
  'pink',
  'lightskyblue',
  'lightskyblue',
  'plum',
  'pink',
  'pink',
  'lightskyblue',
  'pink',
  'lightskyblue',
  'lightskyblue',
  'pink',
  'pink',
  'lightskyblue',
  'pink',
  'plum',
  'lightskyblue',
  'lightskyblue',
  'pink',
  'lightskyblue',
  'lightskyblue',
  'lightskyblue',
  'pink',
  'lightskyblue',
  'lightskyblue',
  'lightskyblue',
  'pink',
  'pink',
  'lightskyblue',
  'plum',
  'pink',
  'lightskyblue',
  'pink',
  'pink',
  'lightskyblue',
  'pink',
  'pink',
  'pink',
  'lightskyblue',
  'lightskyblue',
  'pink',
  'lightskyblue',
  'plum',
  'pink',
  'lightskyblue',
  'lightskyblue',
  'lightskyblue',
  'lightskyblue',
  'pink',
  'pink',
  'pink',
  'lightskyblue',
  'pink',
  'lightskyblue',
  'lightskyblue',
  'lightskyblue',
  'pink',
  'pink',
  'p

In [188]:
liste = [1, 2, 3]

[[1, 2, 3] * 5]

[[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]]

In [52]:
df_nat

,sexe,preusuel,annais,nombre
0,1,_prenoms_rares,1900,1250
1,1,_prenoms_rares,1901,1342
2,1,_prenoms_rares,1902,1330
3,1,_prenoms_rares,1903,1286
4,1,_prenoms_rares,1904,1430
...,...,...,...,...
667358,2,zyneb,2016,6
667359,2,zyneb,2017,6
667360,2,zyneb,2018,5
667361,2,zyneb,2019,7


In [104]:
df_compare.reset_index()

,preusuel,last_10,last_3
0,_prenoms_rares,24152,25526.0
1,gabriel,5233,4968.0
2,leo,4869,4781.0
3,lucas,4763,3657.0
4,raphael,4594,4551.0
...,...,...,...
9836,djamel-eddine,0,NaN
9837,tran,0,NaN
9838,jean-maxime,0,NaN
9839,jean-mathis,0,NaN


In [84]:
# df_nat[df_nat.preusuel.isin(df_compare.index.values)].groupby(['preusuel', 'sexe']).agg(preusuel=('preusuel', 'first'))
# ser_sexe_color = df_nat[df_nat.preusuel.isin(df_compare.index.values)].groupby('preusuel').sexe.nunique()
ser_sexe_color = df_nat[df_nat.preusuel.isin(df_compare.index.values)].groupby('preusuel').agg(nsexes=('sexe', 'nunique'), sexe=('sexe', 'min'))
ser_sexe_color.sample(30)

,nsexes,sexe,fsexe
preusuel,,,
ariel,2,2,1
stefanu,1,1,1
thibo,1,1,1
millan,1,1,1
chamsdine,1,1,1
timour,1,1,1
mocktar,1,1,1
tyson,1,1,1
fouad,1,1,1


In [66]:
ser_sexe_color.loc[df_compare.index.values]

preusuel
_prenoms_rares    2
gabriel           2
leo               2
lucas             2
raphael           2
                 ..
djamel-eddine     1
tran              2
jean-maxime       1
jean-mathis       1
roxan             1
Name: sexe, Length: 9841, dtype: int64

In [48]:
df_compare.head()

,last_10,last_3
preusuel,,
_prenoms_rares,24152,25526.0
gabriel,5233,4968.0
leo,4869,4781.0
lucas,4763,3657.0
raphael,4594,4551.0


## Passé

In [49]:
target_year = 1991
nb_annees = 3
n_display = 50
df_last_years = df_nat[(df_nat.annais.isin([year for year in range(target_year - 1, target_year + 2)])) & (df_nat.sexe == 1)]
df_last_years.groupby(['preusuel']).agg({'nombre' : sum}).sort_values(by='nombre', ascending=False).head(n_display)

,nombre
preusuel,
kevin,37396
thomas,29391
_prenoms_rares,26376
nicolas,26075
alexandre,26032
julien,25991
maxime,24970
anthony,23019
romain,21620
